## VQE
Tenemos 
$f(x_0, x_1,x_2) = 2x_0 - 4 x_0 x_1 + 3 x_1 x_2$

Encontrar los valores binarios de las equis que minimice la función, el valor de mínima energía.

| $x_0$ |  $x_1$ |  $x_2$ |  $f(x_{0,1,2})$ |
|-------|--------|--------|-----------------|
|0|0|0|0|
|0|0|1|0|
|0|1|0|0|
|0|1|1|3|
|1|0|0|2|
|1|0|1|2|
|1|1|0|**-2**|
|1|1|1|1|
 
Cambiamos $f(x)$ por $g(z)$ donde z sean coeficientes binarios pero de valor 1 y -1. Se logra con la conversión $x_i = (1-z_i) /2$
 
Esto que se obtiene representa un circuito cuántico donde los subíndices de z definen el canal donde se hará la operación z, esto es:

$
  g(z_0, z_1, z_2) = \frac{3}{4} + \frac{1}{4}z_1 - \frac{3}{4}z_2 - z_0 z_1 + \frac{3}{4}z_1 z_2 
$

A continuación, este código en Python implementa un algoritmo de optimización cuántica para maximizar una función llamada "value", la cual es definida como una combinación lineal de valores de expectación calculados en distintos circuitos cuánticos. El objetivo es encontrar los valores óptimos de los parámetros "w" que maximizan el valor de la función "value".

Para esto se utiliza el paquete PennyLane, en el cual se define un dispositivo cuántico con "variables" qubits y se definen cinco circuitos cuánticos "circ1" a "circ5", los cuales son evaluados con los parámetros "w" para calcular los valores de expectación necesarios para definir "value".

Luego se define la función "solucion" la cual, dadas los valores óptimos de "w", utiliza un circuito cuántico similar a "phi" para calcular las mediciones de los valores de Pauli Z de cada qubit.

Finalmente, se utiliza un algoritmo de descenso de gradiente para optimizar los parámetros "w" y se imprime el valor de "value" en cada iteración, cada 50 iteraciones.

In [1]:
import pennylane as qml
from pennylane import numpy as np

variables = 3
dev = qml.device("default.qubit", wires= variables, shots = 1000)

def phi(w): 
    qml.RX(w[0], wires=0)
    qml.RX(w[1], wires=1)
    qml.RX(w[2], wires=2)

@qml.qnode(dev)
def circ1(w):
    phi(w)
    return qml.expval(qml.Identity(wires=0) @ qml.Identity(wires=1) @ qml.Identity(wires=2))

@qml.qnode(dev)
def circ2(w):
    phi(w)
    return qml.expval(qml.Identity(wires=0) @ qml.PauliZ(wires=1) @ qml.Identity(wires=2))

@qml.qnode(dev)
def circ3(w):
    phi(w)
    return qml.expval(qml.Identity(wires=0) @ qml.Identity(wires=1) @ qml.PauliZ(wires=2))

@qml.qnode(dev)
def circ4(w):
    phi(w)
    return qml.expval(qml.PauliZ(wires=0) @ qml.PauliZ(wires=1) @ qml.Identity(wires=2))

@qml.qnode(dev)
def circ5(w):
    phi(w)
    return qml.expval(qml.Identity(wires=0) @ qml.PauliZ(wires=1) @ qml.PauliZ(wires=2))

def value(w):
    return 3*circ1(w)/4 + circ2(w)/4 - 3*circ3(w)/4 - circ4(w) + 3*circ5(w)/4

w = qml.numpy.random.rand(3) * 2 * 3.14

gradient_fn_w = qml.grad(value, argnum=0)

lr=0.005
for epoch in range(501):
    w = w -lr*gradient_fn_w(w)
    if epoch % 50 == 0:
        print("epoch", epoch)
        print("loss", value(w))

@qml.qnode(qml.device('default.qubit', wires = variables, shots =1))
def solucion(w):
    phi(w)
    return [qml.sample(qml.PauliZ(i)) for i in range(3)]

solucion(w)

epoch 0
loss 0.2929999999999999
epoch 50
loss 0.1825000000000001
epoch 100
loss 0.13700000000000012
epoch 150
loss 0.08450000000000024
epoch 200
loss 0.057499999999999996
epoch 250
loss 0.052999999999999936
epoch 300
loss 0.03850000000000009
epoch 350
loss -0.0004999999999999449
epoch 400
loss -0.008500000000000063
epoch 450
loss 0.0375000000000002
epoch 500
loss 0.0050000000000001155


tensor([[ 1],
        [ 1],
        [-1]], dtype=int64, requires_grad=True)

En el código anterior, el ansatz utilizado es la función "phi", la cual es un circuito cuántico que consiste en aplicaciones de una rotación RX en cada uno de los tres qubits. Específicamente, la función "phi(w)" aplica un RX(w[0]) en el primer qubit, un RX(w[1]) en el segundo qubit, y un RX(w[2]) en el tercer qubit. Los parámetros "w" especifican los ángulos correspondientes para cada qubit.

Es importante destacar que el ansatz utilizado se encuentra dentro de cada una de las funciones "circ1" a "circ5", por lo que el ansatz será utilizado durante el proceso de optimización mediante descenso de gradiente para encontrar los valores óptimos de los parámetros "w" que maximicen la función "value".